## TP3: Introducción al aprendizaje automático

In [2]:
import pandas as pd

In [3]:
Datos = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2020_Mentoria_LluviasExtremas/master/dataset/datos_diarios_cordoba.csv", parse_dates = True, index_col="fecha")
id_Datos = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2020_Mentoria_LluviasExtremas/master/dataset/metadatos_estaciones.csv")

De los datasets analizados en los TPs anteriores, ahora nos vamos a enfocar en:

- La Florida
- Marcos Juarez

Y vamos a usar como variable que queremos predecir, la PMDA que han calculado para cada año hidrológico (del 1/julio al 30/junio).

Las demás variables (como viento, temperatura, etc) las vamos a resumir para cada año hidrológico en la media (se pueden agregar más estadísticos pero así es suficiente).

##### Actividades:

##### OBJETIVO: Predecir Precipitación Máxima Diaria Anual (PMDA) para los próximos 5 años

1) De los dataset que han trabajado hasta ahora, agreguen más features al DataFrame **"Datos"** (Por lo general, más features tienden a dar mejores predicciones).

2) Además de las features del punto anterior, les propongo agregar nuevas features moviendo las originales tantos "pasos de tiempo" como features nuevas quieran generar.

3) Curar el dataset (si fuera necesario).

4) Aplicar técnicas de selección y extraccion de features. Armar un nuevo dataset con aquellas features más significativas.

5) Dividir el dataset (training, validation, test).. Recuerden que queremos predecir la PMDA para los próximos 5 años.

6) Analizar y elegir 3 modelos para REGRESIÓN. Entrenarlos y analizar resultados.

7) Evaluar predicciones de los diferentes modelos.

8) ¿Qué estadísticos utilizaron para evaluar las regresiones? Justifiquen su elección.

In [6]:
full_dataset = pd.read_csv('./datasetTP2/datos_diarios_cordoba_terra_chirps.csv')

In [47]:
dataset = full_dataset[(full_dataset['nombre'] == 'La Florida') | (full_dataset['nombre'] == 'Marcos Juarez INTA')]
dataset.loc[:, 'fecha'] = pd.to_datetime(dataset['fecha'])
dataset.reset_index(inplace=True)
dataset = dataset.drop(['Unnamed: 0'], axis=1)

In [48]:
def get_hydro_year(date_, start_month=7):
    year = date_.strftime('%Y')
    month = date_.strftime('%m')
    offset = 1 if int(month) >= start_month - 1 else 0
    hydro_year = int(year) + offset - 1
    return hydro_year

def add_hydro_year(dataset):
    dataset = dataset.copy()
    dataset.reset_index(inplace=True)
    dataset['hydro_year'] = dataset['fecha'].apply(get_hydro_year)
    return dataset

In [49]:
dataset = add_hydro_year(dataset)

In [52]:
dataset.tail(200)

,level_0,index,fecha,ano,mes,omm_id,nombre,prcp,tmax,tmin,...,vmax_d,vmax_f,vmed,caudal,ETreal_mm,HumedadSuelo_mm,RadSolar_Wpm2,PresionVapor_hPa,IndicePalmer_porc,hydro_year
41158,41158,84631,2018-05-13,2018,5,1,La Florida,0.0,16.4,5.5,...,NaN,NaN,3.88,NaN,64.0,162.0,108.4,9.33,-1.67,2017
41159,41159,84632,2018-05-14,2018,5,1,La Florida,0.0,16.4,5.5,...,NaN,NaN,3.88,NaN,64.0,162.0,108.4,9.33,-1.67,2017
41160,41160,84633,2018-05-15,2018,5,1,La Florida,0.0,16.4,5.5,...,NaN,NaN,3.88,NaN,64.0,162.0,108.4,9.33,-1.67,2017
41161,41161,84634,2018-05-16,2018,5,1,La Florida,0.0,16.4,5.5,...,NaN,NaN,3.88,NaN,64.0,162.0,108.4,9.33,-1.67,2017
41162,41162,84635,2018-05-17,2018,5,1,La Florida,0.0,16.4,5.5,...,NaN,NaN,3.88,NaN,64.0,162.0,108.4,9.33,-1.67,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41353,41353,84826,2018-11-24,2018,11,1,La Florida,0.0,24.4,10.5,...,NaN,NaN,5.38,NaN,114.5,68.0,258.1,12.26,1.29,2018
41354,41354,84827,2018-11-25,2018,11,1,La Florida,0.0,24.4,10.5,...,NaN,NaN,5.38,NaN,114.5,68.0,258.1,12.26,1.29,2018
41355,41355,84828,2018-11-26,2018,11,1,La Florida,21.5,24.4,10.5,...,NaN,NaN,5.38,NaN,114.5,68.0,258.1,12.26,1.29,2018
41356,41356,84829,2018-11-27,2018,11,1,La Florida,94.0,24.4,10.5,...,NaN,NaN,5.38,NaN,114.5,68.0,258.1,12.26,1.29,2018
